# RAG로 AI소믈리에 wine pairing

In [5]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [6]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM을 통한 요리 정보 해석
- 이미지 -> 맛과 풍미 (image to text)
- 입력 : 요리 이미지(url)
- 출력 : 요리명, 요리에 대한 풍미 설명
- 함수로 정의한 다음, RunnableLambda로 실행

In [10]:
# 와인과 페어링되는 음식
def describe_dish_prompt(input_type):

    prompt = ChatPromptTemplate([
        (
            "system", """
                You are a world-class gourmet and food connoisseur who has tasted cuisines from every culture and era.
                You have an exceptional ability to identify dishes from visual cues alone and to describe flavors with vivid, sensory language.

                Given an image of a dish, do the following:

                1. Identify the most likely name of the dish.
                - If the exact name is uncertain, suggest the closest well-known dish or culinary category.
                2. Briefly describe the origin or culinary style (e.g., Chinese banquet cuisine, Italian rustic cooking, modern fusion).
                3. Describe the flavor profile in rich, evocative detail, including:
                - Dominant tastes (umami, sweetness, saltiness, acidity, bitterness)
                - Aromas and sauces
                - Texture and mouthfeel
                - Balance and depth of flavor
                4. Write as if you have personally eaten this dish many times.

                Tone and style guidelines:
                - Sophisticated, vivid, and expressive
                - Sensory-focused (taste, aroma, texture)
                - Confident but not speculative
                - Avoid mentioning the image itself; speak as a gourmet describing the dish directly

                Output format:

                Dish Name:
                Culinary Style / Origin:
                Flavor Description:

            """),
        HumanMessagePromptTemplate.from_template([
            {"text": """아래의 이미지의 요리에 대한 요리명과 요리의 풍미를 설명해 주세요.
             출력형태 :
             요리명 : 
             풍미 설명 :
             """},
            {"image_url": "{image_url}"} # image_url는 정해져 있음.
        ])
    ])

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.1,
        api_key=GEMINI_API_KEY
    )

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    return chain

In [11]:
from langchain_core.runnables import RunnableLambda

# 함수를 전달인자로 넣기
runnable_1 = RunnableLambda(describe_dish_prompt)

# RunnableLambda를 통한 함수 실행
input_data = {"image_url": "https://postfiles.pstatic.net/MjAyNDAyMTRfMTYx/MDAxNzA3OTEzODI5NDYx.LBvJwuKNKG2Sg6z61m0GwwXClLZ2AtRggm1oq_aGLFkg.tpalMG_Xm2GxJx7JbTfdN_PZ4xXBZky7Y-2lnbokHCcg.JPEG.alswl0224/%EC%A0%84%EA%B0%80%EB%B3%B5_(2).jpg?type=w966"}
response = runnable_1.invoke(input_data)

In [12]:
print(response)

요리명 : 팔보채 (Palbochae)
풍미 설명 :
이 요리는 테이블에 놓이는 순간부터 그 화려하고 풍성한 향으로 미식가의 마음을 사로잡습니다. 바다의 신선함과 흙내음 가득한 버섯의 깊이, 그리고 은은한 생강과 마늘 향이 어우러져 코끝을 간지럽히죠. 모든 재료가 황금빛 윤기 나는 소스에 탐스럽게 뒤덮여 있어, 그 모습만으로도 이미 한 폭의 걸작입니다.

입안에 처음 닿는 순간, 바다의 진귀한 보물들과 귀한 표고버섯이 선사하는 압도적인 감칠맛, 즉 우마미가 혀끝을 지배합니다. 이 깊은 풍미는 새우, 가리비, 그리고 촉촉한 고급 생선살에서 우러나오는 섬세하고 자연스러운 단맛과 절묘하게 균형을 이룹니다. 소스는 단순한 양념이 아닌, 간장과 굴 소스의 농후한 풍미를 기반으로 한 걸작입니다. 은근한 단맛이 더해져 감칠맛을 증폭시키고, 자극적이지 않으면서도 미묘하게 감도는 향신료의 온기가 미각을 부드럽게 감싸 안습니다. 벨벳처럼 부드러운 소스는 입안 가득 충만한 만족감을 주면서도 전혀 부담스럽지 않게 모든 재료를 완벽하게 감싸 안습니다.

식감 또한 이 요리의 백미입니다. 생선살은 더할 나위 없이 부드러워 입안에서 사르르 녹아내리고, 칼집을 넣어 섬세하게 손질된 오징어는 쫄깃하면서도 야들야들한 탄력을 선사합니다. 가리비는 비단처럼 매끄러운 식감을 자랑하며, 연근은 예상치 못한 아삭함으로 풍성한 소스 속에서 상큼한 존재감을 드러냅니다. 두툼한 표고버섯은 씹을 때마다 묵직한 육즙과 함께 흙의 깊은 풍미를 터뜨리며 소스를 한껏 머금어 더욱 진한 맛을 내고, 초록빛 완두콩은 톡 터지는 신선함으로 입안을 리프레시해 줍니다. 부드러움, 쫄깃함, 아삭함 등 다채로운 식감의 조화가 매 순간 새로운 즐거움을 선사하며, 한입 한입이 미각의 탐험과도 같습니다.

이 팔보채는 바다의 풍요로움과 땅의 보물이 만나 빚어낸 깊이와 균형이 완벽한 조화를 이루는, 맛과 향, 식감 모든 면에서 잊을 수 없는 미식 경험을 선사하는 진정한 축제의 요리입니다.


# 요리에 가장 잘 어울리는 와인 top 5 검색
- pinecone 벡터 db 저장되어 있음
- index : wine-review2
- namespace : wine-review2-namespace

In [ ]:
# 사용자 프롬프트 vector화, 유사도 높은 top k개 문서 검색
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

In [ ]:
# 요리에 대한 풍미(설명)이 들어오면,
# 벡터 DB에 인덱싱할 때 사용한 임베딩 모델과 동일한 모델로 임베딩 벡터화 수행
embedding_model = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_MODEL
)

# Pinecone 벡터 db 객체 생성
vector_db = PineconeVectorStore(
    index_name=PINECONE_INDEX_NAME,
    embedding=embedding_model,
    namespace=PINECONE_NAMESPACE
)

# 벡터 DB에서 유사도 높은 top k개 문서 검색
query = response
print("사용자 프롬프트:", query)
print("-"*50)
vector_db.similarity_search(query, k=5)

사용자 프롬프트: 요리명 : 팔보채 (Palbochae)
풍미 설명 :
이 요리는 테이블에 놓이는 순간부터 그 화려하고 풍성한 향으로 미식가의 마음을 사로잡습니다. 바다의 신선함과 흙내음 가득한 버섯의 깊이, 그리고 은은한 생강과 마늘 향이 어우러져 코끝을 간지럽히죠. 모든 재료가 황금빛 윤기 나는 소스에 탐스럽게 뒤덮여 있어, 그 모습만으로도 이미 한 폭의 걸작입니다.

입안에 처음 닿는 순간, 바다의 진귀한 보물들과 귀한 표고버섯이 선사하는 압도적인 감칠맛, 즉 우마미가 혀끝을 지배합니다. 이 깊은 풍미는 새우, 가리비, 그리고 촉촉한 고급 생선살에서 우러나오는 섬세하고 자연스러운 단맛과 절묘하게 균형을 이룹니다. 소스는 단순한 양념이 아닌, 간장과 굴 소스의 농후한 풍미를 기반으로 한 걸작입니다. 은근한 단맛이 더해져 감칠맛을 증폭시키고, 자극적이지 않으면서도 미묘하게 감도는 향신료의 온기가 미각을 부드럽게 감싸 안습니다. 벨벳처럼 부드러운 소스는 입안 가득 충만한 만족감을 주면서도 전혀 부담스럽지 않게 모든 재료를 완벽하게 감싸 안습니다.

식감 또한 이 요리의 백미입니다. 생선살은 더할 나위 없이 부드러워 입안에서 사르르 녹아내리고, 칼집을 넣어 섬세하게 손질된 오징어는 쫄깃하면서도 야들야들한 탄력을 선사합니다. 가리비는 비단처럼 매끄러운 식감을 자랑하며, 연근은 예상치 못한 아삭함으로 풍성한 소스 속에서 상큼한 존재감을 드러냅니다. 두툼한 표고버섯은 씹을 때마다 묵직한 육즙과 함께 흙의 깊은 풍미를 터뜨리며 소스를 한껏 머금어 더욱 진한 맛을 내고, 초록빛 완두콩은 톡 터지는 신선함으로 입안을 리프레시해 줍니다. 부드러움, 쫄깃함, 아삭함 등 다채로운 식감의 조화가 매 순간 새로운 즐거움을 선사하며, 한입 한입이 미각의 탐험과도 같습니다.

이 팔보채는 바다의 풍요로움과 땅의 보물이 만나 빚어낸 깊이와 균형이 완벽한 조화를 이루는, 맛과 향, 식감 모든 면에서 잊을 수 없는 미식 경험을 선사하는 진정한 축제의 요리입니다.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}